# 11.05 - Análisis Temporal Avanzado con Polars

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Avanzado  
**Tiempo estimado:** 45 min

El manejo de fechas en Polars es significativamente más rápido que en Pandas, especialmente para grandes volúmenes de datos temporales.

In [1]:
import polars as pl
from datetime import datetime, timedelta
import numpy as np

# Generar datos temporales (1 año de datos por minuto)
start = datetime(2023, 1, 1)
n = 365 * 24 * 60 
df_time = pl.DataFrame({
    "timestamp": [start + timedelta(minutes=i) for i in range(n)],
    "value": np.random.randn(n).cumsum()
})
df_time.head()

timestamp,value
datetime[μs],f64
2023-01-01 00:00:00,-0.89237
2023-01-01 00:01:00,-0.08179
2023-01-01 00:02:00,0.290745
2023-01-01 00:03:00,0.945977
2023-01-01 00:04:00,1.292584


## 1. Resampling: Group_by_dynamic
En Polars, el remuestreo se realiza mediante `group_by_dynamic`, que es mucho más flexible y rápido.

In [2]:
# Media diaria
df_daily = df_time.group_by_dynamic("timestamp", every="1d").agg([
    pl.col("value").mean().alias("daily_mean"),
    pl.col("value").max().alias("daily_max")
])

df_daily.head()

timestamp,daily_mean,daily_max
datetime[μs],f64,f64
2023-01-01 00:00:00,24.017673,52.971756
2023-01-02 00:00:00,30.851063,65.993163
2023-01-03 00:00:00,112.077981,147.378883
2023-01-04 00:00:00,152.427,179.200384
2023-01-05 00:00:00,153.579751,171.952268


## 2. Window Functions Temporales
Calcular medias móviles es trivial y eficiente con expresiones.

In [3]:
df_rolling = df_time.head(1000).with_columns([
    pl.col("value").rolling_mean(window_size=60).alias("rolling_mean_1h"),
    pl.col("value").rolling_std(window_size=60).alias("rolling_std_1h")
])

df_rolling.head(70)

timestamp,value,rolling_mean_1h,rolling_std_1h
datetime[μs],f64,f64,f64
2023-01-01 00:00:00,-0.89237,null,null
2023-01-01 00:01:00,-0.08179,null,null
2023-01-01 00:02:00,0.290745,null,null
2023-01-01 00:03:00,0.945977,null,null
2023-01-01 00:04:00,1.292584,null,null
…,…,…,…
2023-01-01 01:05:00,10.917989,7.988482,2.459688
2023-01-01 01:06:00,10.696696,8.146676,2.317114
2023-01-01 01:07:00,12.068911,8.312322,2.233454


## 3. Filtrado por rangos de tiempo
Sintaxis limpia para extraer periodos específicos.

In [4]:
# Filtrar solo mañanas (08:00 - 12:00)
df_morning = df_time.filter(
    (pl.col("timestamp").dt.hour() >= 8) & (pl.col("timestamp").dt.hour() <= 12)
)

print(f"Filas en la mañana: {len(df_morning)}")

Filas en la mañana: 109500


## Resumen
- `group_by_dynamic` es la herramienta clave para series temporales.
- Las operaciones `rolling` son vectorizadas y extremadamente rápidas.
- El accessor `.dt` permite manipulaciones complejas sin sacrificar rendimiento.

---

**Anterior:** [11.04 - Polars Lazy API](./11_04_polars_lazy.ipynb)  
